In [0]:
spark.conf.set(
   
)


In [0]:
train_path = "abfss://lakehouse@lab5mri.dfs.core.windows.net/gold/train.parquet"
train_df = spark.read.parquet(train_path).toPandas()

print(train_df.shape)
train_df.head()


(202, 86)


,image_id,label,orig_contrast_mean,orig_contrast_std,orig_dissimilarity_mean,orig_dissimilarity_std,orig_homogeneity_mean,orig_homogeneity_std,orig_ASM_mean,orig_ASM_std,orig_energy_mean,orig_energy_std,orig_correlation_mean,orig_correlation_std,entropy_contrast_mean,entropy_contrast_std,entropy_dissimilarity_mean,entropy_dissimilarity_std,entropy_homogeneity_mean,entropy_homogeneity_std,entropy_ASM_mean,entropy_ASM_std,entropy_energy_mean,entropy_energy_std,entropy_correlation_mean,entropy_correlation_std,gaussian_contrast_mean,gaussian_contrast_std,gaussian_dissimilarity_mean,gaussian_dissimilarity_std,gaussian_homogeneity_mean,gaussian_homogeneity_std,gaussian_ASM_mean,gaussian_ASM_std,gaussian_energy_mean,gaussian_energy_std,gaussian_correlation_mean,gaussian_correlation_std,sobel_contrast_mean,sobel_contrast_std,...,sobel_energy_mean,sobel_energy_std,sobel_correlation_mean,sobel_correlation_std,prewitt_contrast_mean,prewitt_contrast_std,prewitt_dissimilarity_mean,prewitt_dissimilarity_std,prewitt_homogeneity_mean,prewitt_homogeneity_std,prewitt_ASM_mean,prewitt_ASM_std,prewitt_energy_mean,prewitt_energy_std,prewitt_correlation_mean,prewitt_correlation_std,hessian_contrast_mean,hessian_contrast_std,hessian_dissimilarity_mean,hessian_dissimilarity_std,hessian_homogeneity_mean,hessian_homogeneity_std,hessian_ASM_mean,hessian_ASM_std,hessian_energy_mean,hessian_energy_std,hessian_correlation_mean,hessian_correlation_std,gabor_contrast_mean,gabor_contrast_std,gabor_dissimilarity_mean,gabor_dissimilarity_std,gabor_homogeneity_mean,gabor_homogeneity_std,gabor_ASM_mean,gabor_ASM_std,gabor_energy_mean,gabor_energy_std,gabor_correlation_mean,gabor_correlation_std
0,19 no.jpg,0,418.999626,106.862960,8.619529,1.222411,0.467391,0.024547,0.077020,0.003097,0.277469,0.005564,0.911129,0.022536,165.675485,47.837449,8.276918,1.308316,0.328678,0.021652,0.051155,0.001603,0.226148,0.003526,0.990668,0.002709,133.392920,42.417396,5.301117,0.922597,0.520376,0.026886,0.078496,0.002164,0.280144,0.003847,0.969997,0.009502,718.003209,146.911125,...,0.283336,0.007117,0.740114,0.052721,653.459127,145.588109,10.971204,1.461889,0.442086,0.022067,0.079823,0.004337,0.282427,0.007593,0.749470,0.055386,9219.290016,1497.988859,36.154078,5.874466,0.858221,0.023037,0.404468,0.016905,0.635839,0.013284,0.701164,0.048334,192.987537,104.725518,5.776386,2.399937,0.480972,0.090147,0.074006,0.011819,0.271240,0.020867,0.575132,0.230483
1,Y105.jpg,1,992.417326,287.617513,12.592945,2.191831,0.435541,0.017265,0.119426,0.002194,0.345566,0.003170,0.936231,0.018479,152.574593,41.385597,7.317863,1.028119,0.384149,0.015927,0.083288,0.001547,0.288585,0.002678,0.992633,0.002012,378.780000,123.948194,8.942755,1.672818,0.442970,0.022732,0.103958,0.001826,0.322413,0.002829,0.972728,0.008930,1316.626824,341.422201,...,0.320273,0.003440,0.717966,0.072737,1347.183041,364.712503,15.776931,2.547497,0.414424,0.017736,0.102166,0.002271,0.319614,0.003549,0.730901,0.072478,9617.884333,1708.462923,37.717193,6.699855,0.852092,0.026274,0.383827,0.018945,0.619350,0.015241,0.698681,0.053377,142.526726,78.134252,4.616986,1.948193,0.510316,0.095906,0.077514,0.011170,0.277741,0.019337,0.594090,0.222463
2,Y42.jpg,1,383.625832,125.168821,8.460736,1.623895,0.403963,0.029416,0.056603,0.002296,0.237866,0.004824,0.940497,0.019596,175.698488,51.370943,8.966835,1.402025,0.272000,0.024065,0.026558,0.000828,0.162946,0.002532,0.988661,0.003345,252.891338,83.358352,7.307264,1.402623,0.435296,0.032905,0.064282,0.001612,0.253519,0.003172,0.960605,0.013048,695.437306,207.746708,...,0.211946,0.004841,0.823217,0.053510,708.395584,213.337601,12.026171,2.227418,0.366476,0.028661,0.049152,0.002270,0.221644,0.005120,0.826126,0.053080,9799.379904,1782.449832,38.428941,6.989999,0.849301,0.027411,0.376439,0.019423,0.613343,0.015789,0.696377,0.055161,169.461178,92.779162,5.901968,2.560368,0.434546,0.106174,0.053594,0.006864,0.231059,0.014349,0.652587,0.190151
3,Y91.jpg,1,38.281758,12.554805,3.081662,0.529044,0.486295,0.024

In [0]:
non_feature_cols = ["image_id", "label"]
feature_cols = [c for c in train_df.columns if c not in non_feature_cols]

X = train_df[feature_cols]
y = train_df["label"]

print("Number of features:", X.shape[1])


Number of features: 84


In [0]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.0001)
X_reduced = selector.fit_transform(X)

selected_baseline_features = [
    feature_cols[i] for i in range(len(feature_cols)) if selector.variances_[i] > 0.0001
]

print("Baseline selected features:", len(selected_baseline_features))


Baseline selected features: 61


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# quick split inside the training set for validation
X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(
    X_reduced, y, test_size=0.25, stratify=y, random_state=42
)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_sub, y_train_sub)

baseline_accuracy = accuracy_score(y_val_sub, rf.predict(X_val_sub))
baseline_num_features = X_reduced.shape[1]

baseline_metrics = {
    "baseline_accuracy": float(baseline_accuracy),
    "baseline_num_features": int(baseline_num_features)
}

baseline_metrics


Uploading artifacts:   0%|          | 0/3 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

{'baseline_accuracy': 0.7450980392156863, 'baseline_num_features': 61}

In [0]:
import json

baseline_path = "abfss://lakehouse@lab5mri.dfs.core.windows.net/gold/baseline_metrics.json"

dbutils.fs.put(baseline_path, json.dumps(baseline_metrics), overwrite=True)

print("Saved baseline metrics to:", baseline_path)


Wrote 70 bytes.
Saved baseline metrics to: abfss://lakehouse@lab5mri.dfs.core.windows.net/gold/baseline_metrics.json


In [0]:
%pip install deap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/136.0 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 133.1/136.0 kB 5.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import random
import numpy as np
from deap import base, creator, tools

num_features = len(feature_cols)

# Fitness = accuracy minus penalty
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Binary gene (0 or 1)
toolbox.register("attr_bool", lambda: random.randint(0,1))

# Individual = (binary vector of size num_features)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=num_features)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [0]:
def evaluate(individual):
    selected = [i for i, bit in enumerate(individual) if bit == 1]

    # edge case: no features selected
    if len(selected) == 0:
        return (0.0,)

    X_selected = X.iloc[:, selected]

    X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(
        X_selected, y, test_size=0.25, stratify=y, random_state=42
    )

    clf = RandomForestClassifier(n_estimators=50, random_state=42)
    clf.fit(X_train_sub, y_train_sub)
    acc = accuracy_score(y_val_sub, clf.predict(X_val_sub))

    # penalty for too many features
    penalty = 0.001 * len(selected)

    fitness = acc - penalty
    return (fitness,)


In [0]:
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=20)

N_GEN = 10  # requirement: ≥10 generations

best_ind = None
best_score = -999
import time
start_ga = time.time()

for gen in range(N_GEN):
    offspring = toolbox.select(population, len(population))
    offspring = [toolbox.clone(ind) for ind in offspring]

    # crossover + mutation
    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        tools.cxTwoPoint(ind1, ind2)
        tools.mutFlipBit(ind1, indpb=0.05)
        del ind1.fitness.values, ind2.fitness.values

    # evaluate
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    for ind in invalid_ind:
        ind.fitness.values = toolbox.evaluate(ind)

    # update population
    population[:] = offspring

    for ind in population:
        if ind.fitness.values[0] > best_score:
            best_ind = ind
            best_score = ind.fitness.values[0]

ga_runtime = time.time() - start_ga

print("Best GA fitness:", best_score)


Uploading artifacts:   0%|          | 0/3 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/3 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/3 [00:00<?, ?it/s]

...

Uploading artifacts:   0%|          | 0/3 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/3 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Best GA fitness: 0.7895294117647058


In [0]:
selected_ga_indices = [i for i, bit in enumerate(best_ind) if bit == 1]
selected_ga_features = [feature_cols[i] for i in selected_ga_indices]

ga_accuracy = float(best_score)
ga_num_features = len(selected_ga_features)

ga_metrics = {
    "ga_accuracy": ga_accuracy,
    "ga_num_features": ga_num_features,
    "ga_runtime_seconds": ga_runtime,
    "selected_features": selected_ga_features
}

ga_metrics


{'ga_accuracy': 0.7895294117647058,
 'ga_num_features': 34,
 'ga_runtime_seconds': 991.3699250221252,
 'selected_features': ['orig_contrast_std',
  'orig_energy_std',
  'orig_correlation_mean',
  'entropy_contrast_std',
  'entropy_dissimilarity_mean',
  'entropy_dissimilarity_std',
  'entropy_homogeneity_mean',
  'entropy_homogeneity_std',
  'entropy_energy_std',
  'entropy_correlation_mean',
  'gaussian_contrast_mean',
  'gaussian_ASM_std',
  'gaussian_correlation_mean',
  'sobel_contrast_mean',
  'sobel_dissimilarity_std',
  'sobel_homogeneity_std',
  'sobel_ASM_std',
  'sobel_energy_mean',
  'sobel_correlation_mean',
  'sobel_correlation_std',
  'prewitt_contrast_std',
  'prewitt_energy_mean',
  'prewitt_correlation_mean',
  'hessian_contrast_mean',
  'hessian_contrast_std',
  'hessian_homogeneity_mean',
  'hessian_ASM_mean',
  'hessian_correlation_std',
  'gabor_contrast_mean',
  'gabor_contrast_std',
  'gabor_dissimilarity_std',
  'gabor_ASM_std',
  'gabor_energy_std',
  'gabor_co

In [0]:
import json

selected_features_path = "abfss://lakehouse@lab5mri.dfs.core.windows.net/gold/selected_features.json"
ga_metrics_path = "abfss://lakehouse@lab5mri.dfs.core.windows.net/gold/ga_metrics.json"

dbutils.fs.put(selected_features_path, json.dumps(selected_ga_features), overwrite=True)
dbutils.fs.put(ga_metrics_path, json.dumps(ga_metrics), overwrite=True)

print("Saved:")
print(" -", selected_features_path)
print(" -", ga_metrics_path)


Wrote 826 bytes.
Wrote 948 bytes.
Saved:
 - abfss://lakehouse@lab5mri.dfs.core.windows.net/gold/selected_features.json
 - abfss://lakehouse@lab5mri.dfs.core.windows.net/gold/ga_metrics.json


In [0]:
display(dbutils.fs.ls("abfss://lakehouse@lab5mri.dfs.core.windows.net/"))




path,name,size,modificationTime
abfss://lakehouse@lab5mri.dfs.core.windows.net/bronze/,bronze/,0,1764432432000
abfss://lakehouse@lab5mri.dfs.core.windows.net/gold/,gold/,0,1764441939000
abfss://lakehouse@lab5mri.dfs.core.windows.net/raw/,raw/,0,1763921781000
abfss://lakehouse@lab5mri.dfs.core.windows.net/silver/,silver/,0,1764437773000
